<a href="https://colab.research.google.com/github/drysmakhanova/Robt403/blob/main/FKLab6_DR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np 
import keras
from keras.models import Sequential
from keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.optimizers import Adam
import tensorflow.keras.optimizers 
from keras.utils import np_utils 
from keras import backend as K 
import pandas as pd
from sklearn.model_selection import train_test_split
import random


In [3]:
data = pd.read_csv("dict1.csv", header = None, names = ["Angles", "XY"])


In [4]:
train = data['Angles'].to_numpy()
labels = data['XY'].to_numpy()

Here we use for labels translation and rotation values and keep only X,Y coordinaties and Rotation values.

In [5]:
X = list()
Y = list()
for i in range(len(train)):
    train[i] = train[i].strip('(').strip(')')
    result = [float(val) for val in train[i].split(',')]
    X.append(result)
    labels[i] = labels[i].strip('(array([').strip(']))')
    labels[i] = labels[i].replace(' ', '')
    labels[i] = labels[i].replace('  ', '')
    labels[i] = labels[i].replace(',0.25]),array([0.,0.,', ',')
    labels[i] = labels[i].replace(',2.5000e-01]),array([0.,0.,', ',')
    labels[i] = labels[i].replace(',0.25]),array([0.e+00,0.e+00,', ',')
    result = [float(val) for val in labels[i].split(',')]
    result.pop(2)
    result[2] = 2*np.arccos(result[2])
    result[2] = round(result[2], 4)
    Y.append(result)

So X values are 5  joint angles values of the robot and Y are labels: X,Y,Rotation

Shuffling lists and keeping X-Y order

In [7]:
temp = list(zip(X, Y))
random.shuffle(temp)
X,Y = zip(*temp)

In [8]:
X_train, X_test, y_train, y_test = train_test_split(np.asarray(X), np.asarray(Y), test_size=0.25)

In [9]:
print(np.shape(X_train))
print(np.shape(y_train))
print(np.shape(X_test))
print(np.shape(y_test))

(375, 5)
(375, 3)
(125, 5)
(125, 3)


In [139]:
def rmse(y_true, y_pred):
    return K.sqrt(K.mean(K.square(y_pred - y_true)))

model = Sequential()
model.add(Dense(32, input_dim =5, activation = 'elu'))
model.add(Dense(64, activation = 'elu'))
model.add(Dense(128, activation = 'elu'))
model.add(Dense(256, activation = 'elu'))
model.add(Dense(128, activation = 'elu'))
model.add(Dense(64, activation = 'elu'))
model.add(Dense(32, activation = 'elu'))
model.add(Dense(3, activation='elu'))
model.compile(loss=rmse, optimizer=Adam(0.001))

In [140]:
model.summary()

Model: "sequential_20"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_87 (Dense)             (None, 32)                192       
_________________________________________________________________
dense_88 (Dense)             (None, 64)                2112      
_________________________________________________________________
dense_89 (Dense)             (None, 128)               8320      
_________________________________________________________________
dense_90 (Dense)             (None, 256)               33024     
_________________________________________________________________
dense_91 (Dense)             (None, 128)               32896     
_________________________________________________________________
dense_92 (Dense)             (None, 64)                8256      
_________________________________________________________________
dense_93 (Dense)             (None, 32)              

In [141]:
model.fit(X_train, y_train, epochs = 20)

Epoch 1/20
12/12 [==============================] - 1s 3ms/step - loss: 1.6254
Epoch 2/20
12/12 [==============================] - 0s 3ms/step - loss: 0.5946
Epoch 3/20
12/12 [==============================] - 0s 3ms/step - loss: 0.4001
Epoch 4/20
12/12 [==============================] - 0s 4ms/step - loss: 0.3004
Epoch 5/20
12/12 [==============================] - 0s 4ms/step - loss: 0.2671
Epoch 6/20
12/12 [==============================] - 0s 3ms/step - loss: 0.2546
Epoch 7/20
12/12 [==============================] - 0s 4ms/step - loss: 0.2483
Epoch 8/20
12/12 [==============================] - 0s 3ms/step - loss: 0.2430
Epoch 9/20
12/12 [==============================] - 0s 4ms/step - loss: 0.2463
Epoch 10/20
12/12 [==============================] - 0s 4ms/step - loss: 0.2499
Epoch 11/20
12/12 [==============================] - 0s 4ms/step - loss: 0.2494
Epoch 12/20
12/12 [==============================] - 0s 3ms/step - loss: 0.2508
Epoch 13/20
12/12 [==============================

In [142]:
scores = model.evaluate(X_test, y_test, verbose=0) 
print("RMSE: %.2f" % (scores))

RMSE: 0.23


In [143]:
print(model.predict(X_test[10].reshape(1,5)))
print(y_test[10])

[[3.0291338 1.6235226 1.1633207]]
[3.1107 1.5675 1.0796]


Elu and Selu show the best approximation